In [14]:
from transformers import pipeline, BertTokenizer, BertForMaskedLM
from transformers.pipelines.fill_mask import FillMaskPipeline
import torch

In [15]:
# torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch_device = 'cpu'

print(f"Torch Device: {torch_device}")

language = 'en'

# EN
model_name = "bert-large-uncased"
# model_name = "bert-base-uncased"

# FA
## Lite BERT
# model_name = "HooshvareLab/albert-fa-zwnj-base-v2"

## BERT 
# model_name = "HooshvareLab/bert-fa-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_name)

if language == 'en':
    model = BertForMaskedLM.from_pretrained(model_name).to(torch_device)

else:
    model = BertForMaskedLM.from_pretrained(model_name)

print(f"{language} Model Loaded ...")

unmasker = pipeline("fill-mask", model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


en Model Loaded ...
Torch Device: cpu
en Model Loaded ...


In [16]:
len(tokenizer.get_vocab())

30522

30522

In [17]:
text = "The capital of Iran is [MASK]."
text = "Yesterday, I played [MASK] in the [MASK]."
text = "I'm studying [MASK] learning in my computer class."
text = "I'm a very [MASK] player in football."
text = "He drived a [MASK]."
text = "I love playing [MASK]."
text = "I am [MASK] english very [MASK]. "

# text = "امروز در استادیوم آزادی، تیم ملی [MASK] ایران و سوریه مسابقه می‌دهند."
# text = "امروز در استادیوم آزادی [ماسک] ملی ایران و روسیه مسایقه می‌دهند."
# text = "پس از سال‌ها تلاش رازی موفق به [ماسک] الکل شد. این دانشمند تیرانی باعث افتخار در تاریخ [ماسک] است."
# text = "اهل کدام کشور هستی[ماسک]"
# text = text.replace("[ماسک]", "[MASK]")

unmasker(text, )

[[{'score': 0.5726480484008789,
   'token': 4092,
   'token_str': 's p e a k i n g',
   'sequence': '[CLS] i am speaking english very [MASK]. [SEP]'},
  {'score': 0.30466437339782715,
   'token': 4083,
   'token_str': 'l e a r n i n g',
   'sequence': '[CLS] i am learning english very [MASK]. [SEP]'},
  {'score': 0.018297268077731133,
   'token': 2635,
   'token_str': 't a k i n g',
   'sequence': '[CLS] i am taking english very [MASK]. [SEP]'},
  {'score': 0.017190691083669662,
   'token': 5702,
   'token_str': 's t u d y i n g',
   'sequence': '[CLS] i am studying english very [MASK]. [SEP]'},
  {'score': 0.011624318547546864,
   'token': 2478,
   'token_str': 'u s i n g',
   'sequence': '[CLS] i am using english very [MASK]. [SEP]'}],
 [{'score': 0.8824039697647095,
   'token': 2092,
   'token_str': 'w e l l',
   'sequence': '[CLS] i am [MASK] english very well. [SEP]'},
  {'score': 0.023844681680202484,
   'token': 2411,
   'token_str': 'o f t e n',
   'sequence': '[CLS] i am [MASK

[[{'score': 0.5726480484008789,
   'token': 4092,
   'token_str': 's p e a k i n g',
   'sequence': '[CLS] i am speaking english very [MASK]. [SEP]'},
  {'score': 0.30466437339782715,
   'token': 4083,
   'token_str': 'l e a r n i n g',
   'sequence': '[CLS] i am learning english very [MASK]. [SEP]'},
  {'score': 0.018297268077731133,
   'token': 2635,
   'token_str': 't a k i n g',
   'sequence': '[CLS] i am taking english very [MASK]. [SEP]'},
  {'score': 0.017190691083669662,
   'token': 5702,
   'token_str': 's t u d y i n g',
   'sequence': '[CLS] i am studying english very [MASK]. [SEP]'},
  {'score': 0.011624318547546864,
   'token': 2478,
   'token_str': 'u s i n g',
   'sequence': '[CLS] i am using english very [MASK]. [SEP]'}],
 [{'score': 0.8824039697647095,
   'token': 2092,
   'token_str': 'w e l l',
   'sequence': '[CLS] i am [MASK] english very well. [SEP]'},
  {'score': 0.023844681680202484,
   'token': 2411,
   'token_str': 'o f t e n',
   'sequence': '[CLS] i am [MASK

# Stanza

spaCy's tokenization is non-destructive, so it always represents the original input text and never adds or deletes anything. This is kind of a core principle of the Doc object: you should always be able to reconstruct and reproduce the original input text.

## Setup

In [18]:
import stanza
import spacy
import spacy_stanza

In [19]:
if language == 'fa':
    stanza.install_corenlp()
    stanza.download('en')
    nlp = spacy_stanza.load_pipeline("en")

elif language == 'en':
    spacy.prefer_gpu()
    nlp = spacy.load("en_core_web_lg")


In [31]:
# tokens = tokenizer.convert_ids_to_tokens(tokenizer(text)['input_ids'])
# tokenizer.convert_tokens_to_string(tokens)

# tokenizer.get_vocab()

from spacy.tokens.token import Token
from spacy.tokens.doc import Doc
import editdistance
import pandas as pd

text = "The capitan of Iran is tehran."
text = "i am speeking english very wall."
# text = "Yesterday, I played [MASK] in the [MASK]."
# text = "I'm studying [MASK] learning in my computer class."
# text = "I'm a very [MASK] player in football."
# text = "He drove a cat."
# text = "do you want to watch tv."
# text = "I love playing [MASK]."

text = text.lower()
vocab: dict = tokenizer.get_vocab()
MASK = "[MASK]"

MAX_EDIT_DISTANCE_TO_LEN_RATIO = 0.5
MIN_SCORE = 0.01

### Correct Lexico Typo
print("\n\nCorrect Lexico Typo:\n\n")
print(f"Text: {text}\n\n")
while True:
    some_token_corrected = False
    doc = nlp(text)
    for index in range(len(doc)):
        current_token: Token = doc[index]
        start_char_index = current_token.idx
        end_char_index = start_char_index + len(current_token)

        if current_token.text not in vocab:
            print(f"[{current_token.text}] is not in vocab")

            tokens = list(doc)
            print(f"Tokens: {tokens}")

            masked_text = doc.text[:start_char_index] + MASK + doc.text[end_char_index:]

            predicts = unmasker(masked_text)

            ### Select Token From Predicts
            predicts = pd.DataFrame(predicts)


            predicts['token_str'] = predicts['token_str'].apply(lambda tk: tk.replace(" ", ""))
            predicts['edit_distance_to_len_ratio'] = predicts['token_str'].apply(lambda tk: editdistance.eval(current_token.text, tk) / len(current_token.text))

            print("Predicts: \n")
            print(predicts)

            predicts = predicts[predicts['edit_distance_to_len_ratio'] <= MAX_EDIT_DISTANCE_TO_LEN_RATIO]

            print("Filtered Predicts: \n")
            print(predicts)

            selected_predict = predicts['token_str'].iloc[0]
            ###

            result_text = masked_text.replace(MASK, selected_predict, 1)

            print(f"{current_token.text} -> {selected_predict}")
            print(f"Text: {result_text}\n\n")

            text = result_text
            some_token_corrected = True
            break

    if not some_token_corrected:
        break



Correct Lexico Typo:


Text: i am speeking english very god.


[speeking] is not in vocab
Tokens: [i, am, speeking, english, very, god, .]
Predicts: 

      score  token token_str                         sequence  \
0  0.335115   2019        an        i am an english very god.   
1  0.158579   2025       not       i am not english very god.   
2  0.138133   1996       the       i am the english very god.   
3  0.133903   4092  speaking  i am speaking english very god.   
4  0.053356   1999        in        i am in english very god.   

   edit_distance_to_len_ratio  
0                       0.875  
1                       1.000  
2                       0.875  
3                       0.125  
4                       0.750  
Filtered Predicts: 

      score  token token_str                         sequence  \
3  0.133903   4092  speaking  i am speaking english very god.   

   edit_distance_to_len_ratio  
3                       0.125  
speeking -> speaking
Text: i am speaking english

In [32]:
### Correct Contextual Typo
print("\n\nCorrect Contextual Typo:\n\n")
print(f"Text: {text}\n\n")
while True:
    some_token_corrected = False
    doc = nlp(text)
    for index in range(len(doc)):
        current_token: Token = doc[index]
        start_char_index = current_token.idx
        end_char_index = start_char_index + len(current_token)

        tokens = list(doc)
        print(f"Tokens: {tokens}")

        masked_text = doc.text[:start_char_index] + MASK + doc.text[end_char_index:]

        predicts = unmasker(masked_text)

        ### Select Token From Predicts
        predicts = pd.DataFrame(predicts)

        predicts['token_str'] = predicts['token_str'].apply(lambda tk: tk.replace(" ", ""))
        predicts['edit_distance_to_len_ratio'] = predicts['token_str'].apply(lambda tk: editdistance.eval(current_token.text, tk) / len(current_token.text))

        print("Predicts: \n")
        print(predicts)

        predicts = predicts[(predicts['edit_distance_to_len_ratio'] <= MAX_EDIT_DISTANCE_TO_LEN_RATIO) & (predicts['score'] >= MIN_SCORE)]

        print("Filtered Predicts: \n")
        print(predicts)

        # selected_predict = predicts.loc[0, 'token_str']
        ###
        #
        # result_text = masked_text.replace(MASK, selected_predict, 1)
        #
        # print(f"{current_token.text} -> {selected_predict}")
        # print(f"Text: {result_text}\n\n")
        #
        # text = result_text
        # some_token_corrected = True
        # break

    break
    if not some_token_corrected:
        break



Correct Contextual Typo:


Text: i am speaking english very god.


Tokens: [i, am, speaking, english, very, god, .]
Predicts: 

          score  token token_str                           sequence  \
0  9.999995e-01   1045         i    i am speaking english very god.   
1  1.979739e-07   1998       and  and am speaking english very god.   
2  5.421951e-08   1000         "    " am speaking english very god.   
3  5.051135e-08   2021       but  but am speaking english very god.   
4  2.728065e-08   2026        my   my am speaking english very god.   

   edit_distance_to_len_ratio  
0                         0.0  
1                         3.0  
2                         1.0  
3                         3.0  
4                         2.0  
Filtered Predicts: 

   score  token token_str                         sequence  \
0    1.0   1045         i  i am speaking english very god.   

   edit_distance_to_len_ratio  
0                         0.0  
Tokens: [i, am, speaking, english, very, 

In [22]:
# # tokens = tokenizer.convert_ids_to_tokens(tokenizer(text)['input_ids'])
# # tokens
# # tokenizer.convert_tokens_to_string(tokens)
#
# from spacy.tokens.token import Token
# from spacy.tokens.doc import Doc
#
# text = "پس از سال‌ها تلاش رازی موفق به کسف الکل شد. این دانشمند تیرانی باعث افتخار در تاریخ کور است."
# doc = stanza_nlp(text)
#
#
# # for index in range(len(doc)):
# #     token_str: Token = doc[index]
# #     # print(f"{index}: {token_str.text}")
#
# #     print(doc)
# #     break
#
#
# len(stanza_nlp.vocab.strings)

In [23]:
# text = "The capital of Iran is [MASK]."
# text = "Yesterday, I played [MASK] in the [MASK]."
# text = "I'm studying [MASK] learning in my computer class."
# text = "I'm a very [MASK] player in football."
# text = "He drived a [MASK]."
# text = "I love playing [MASK]."
# text = "I am [MASK] english very [MASK]. "
#
# # text = "امروز در استادیوم آزادی، تیم ملی [MASK] ایران و سوریه مسابقه می‌دهند."
# # text = "امروز در استادیوم آزادی [ماسک] ملی ایران و روسیه مسایقه می‌دهند."
# # text = "پس از سال‌ها تلاش رازی موفق به [ماسک] الکل شد. این دانشمند تیرانی باعث افتخار در تاریخ [ماسک] است."
# # text = "اهل کدام کشور هستی[ماسک]"
# # text = text.replace("[ماسک]", "[MASK]")
#
# unmasker(text, )

In [24]:
# tokens = tokenizer.convert_ids_to_tokens(tokenizer(text)['input_ids'])
# tokens
# tokenizer.convert_tokens_to_string(tokens)


# ContextualSpellCheck Lib

In [25]:
import spacy
import contextualSpellCheck

nlp = spacy.load('en_core_web_lg')
contextualSpellCheck.add_to_pipe(nlp)

In [26]:
text = 'i am speaking english very wall.'
doc = nlp(text)

print(doc._.performed_spellCheck) #Should be True
print(doc._.outcome_spellCheck) #Income was $9.4 million compared to the prior year of $2.7 million.

True
i am speaking this very wall.
True
i am speaking this very wall.
